# Hyperparameters optimization


### Rafal Pronko
rafalpronko @ LinkedIn, Telegram, Twitter, Tungsten


| <img src="czmis.png" width="100">  | <img src="webinterpret.png" width="100">  |  <img src="pm.jpg" width="100"> |
|---|---|---|
| <img src="ynd.png" width="100">  | <img src="cleanride.svg" width="100">  | <img src="cvtimeline.jpg" width="100">  |






## Data

### Santander Customer Transaction Prediction


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/processed/tomek_train.csv")

## Tomek Links


<img src="tomek_links.png" width="400">

In [3]:
df_test = pd.read_csv("../data/processed/train_test.csv")

In [3]:
df.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,target
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914,0
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518,0
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965,0
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996,0
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104,0


In [4]:
df.shape

(158443, 201)

In [5]:
import pandas_profiling

In [6]:
pandas_profiling.ProfileReport(pd.DataFrame(df["target"]))

Number of variables,1
Number of observations,158443
Total Missing (%),0.0%
Total size in memory,1.2 MiB
Average record size in memory,8.0 B
Numeric,0
Categorical,0
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [4]:
from imblearn.under_sampling import RandomUnderSampler

In [5]:
cols = [x for x in df.columns if x not in ["ID_code", "target"]]

In [6]:
rus = RandomUnderSampler()

In [7]:
X, y = rus.fit_resample(df[cols], df["target"])

In [8]:
X.shape

(32014, 200)

In [91]:
import lightgbm as lgb
import warnings
from sklearn.metrics import roc_auc_score
warnings.filterwarnings("ignore")

In [92]:
lgbc = lgb.LGBMClassifier()

In [93]:
lgbc.fit(X, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [94]:
pred = lgbc.predict_proba(df_test[cols])[:,1]

In [95]:
roc_auc_score(df_test["target"], pred)

0.8578124628689625

In [10]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

## GridSearch
```GridSearchCV(estimator, param_grid, scoring=None, fit_params=None, n_jobs=None, iid=’warn’, refit=True, cv=’warn’, verbose=0, pre_dispatch=‘2*n_jobs’, error_score=’raise-deprecating’, return_train_score=’warn’)```

In [97]:
param = {
    'num_leaves':[5, 10, 30, 50, 100],
    'learning_rate':[0.1, 0.01, 0.001],
    'n_estimators': [10, 20, 50, 100]
    
}

In [100]:
gscv = GridSearchCV(lgb.LGBMClassifier(), param_grid=param, n_jobs=1, verbose=1)

In [101]:
gscv.fit(X,y)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:  5.4min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'num_leaves': [5, 10, 30, 50, 100], 'learning_rate': [0.1, 0.01, 0.001], 'n_estimators': [10, 20, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [104]:
gscv.best_params_

{'learning_rate': 0.1, 'n_estimators': 100, 'num_leaves': 50}

In [105]:
pred = gscv.predict_proba(df_test[cols])[:,1]

In [106]:
roc_auc_score(df_test["target"], pred)

0.8639398366762926

## RandomSearch
```RandomizedSearchCV(estimator, param_distributions, n_iter=10, scoring=None, fit_params=None, n_jobs=None, iid=’warn’, refit=True, cv=’warn’, verbose=0, pre_dispatch=‘2*n_jobs’, random_state=None, error_score=’raise-deprecating’, return_train_score=’warn’)```

In [52]:
from scipy.stats import randint, uniform

In [102]:
param = {
    'num_leaves':randint(1, 1000),
    'learning_rate': uniform(),
    'n_estimators': randint(1, 1000)
    
}

In [103]:
rgs = RandomizedSearchCV(lgb.LGBMClassifier(), param_distributions=param, n_jobs=1, verbose=1)

In [107]:
rgs.fit(X,y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 21.9min finished


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1),
          fit_params=None, iid='warn', n_iter=10, n_jobs=1,
          param_distributions={'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x12c5d4160>, 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1339e84a8>, 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1339e8c18>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [110]:
pred = rgs.predict_proba(df_test[cols])[:,1]

In [111]:
roc_auc_score(df_test["target"], pred)

0.8746015272765152

## Hyperopt
- define the objective function to minimize
- the space over which to search
- trials database (optional)
- the search algorithm to use (optional)

In [112]:
from hyperopt import tpe, hp, fmin

In [116]:
def objective_func(param):
    try:
        lgbc = lgb.LGBMClassifier(**param)
        lgbc.fit(X,y)
        y_pred_test = lgbc.predict_proba(df_test[cols])[:,1]
        loss = roc_auc_score(df_test["target"],y_pred_test)
    except Exception as e:
        loss = -1000
        print(e)
    return -loss

### Search space
- hp.choice(label, options)
- hp.uniform(label, low, high)
- hp.normal(label, mu, sigma)
- hp.randint(label, upper)


In [117]:
args = {
        'learning_rate': hp.uniform('learning_rate', 0.03, 1.0),
         'num_leaves':hp.randint("num_leaves", 1000), 
        "n_estimators": hp.randint("n_estimators", 1000),
}

In [118]:
best_classifier = fmin(objective_func,args,
                        algo=tpe.suggest,max_evals=10)

100%|██████████| 10/10 [04:55<00:00, 25.81s/it, best loss: -0.875187046830312]


## Scikit-optimize


In [120]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args

In [122]:
space  = [Integer(1, 1000, name='num_leaves'),
          Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
          Integer(100, 1000, name='n_estimators')]

In [123]:
lgb_c = lgb.LGBMClassifier()

In [155]:
def objective(params):
    try:
        p = {
            "num_leaves": params[0],
            "learning_rate": params[1],
            "n_estimators": params[2],
        }
        lgbc = lgb.LGBMClassifier(**p)
        lgbc.fit(X,y)
        y_pred_test = lgbc.predict_proba(df_test[cols])[:,1]
        loss = roc_auc_score(df_test["target"],y_pred_test)
    except Exception as e:
        loss = -1000
        print(e)
    print(loss)
    return -loss
    
    

In [156]:
from skopt import gp_minimize

In [157]:
res_gp = gp_minimize(objective, space, n_calls=10, random_state=0)

0.8761029392319195
0.8530335845343712
0.6900265234265445
0.8747043429172818
0.873363825459041
0.7816536217166836
0.8768656700923957
0.8392898480670867
0.8518198848321873
0.681228556916248


In [158]:
res_gp.fun

-0.8768656700923957

## Summarize

First model: 0.8578124628689625

GridSearch: 0.8639398366762926

RandomSearch: 0.8746015272765152

Hyperopt: 0.875187046830312

skopt: 0.8768656700923957

## QA


rafalpronko @ LinkedIn, Telegram, Twitter, Tungsten

## References

### Libraries
1. [MOE](https://github.com/Yelp/MOE)
2. [Spearmint](https://github.com/HIPS/Spearmint)
3. [GPyOpt](https://github.com/SheffieldML/GPyOpt)
4. [HyperOpt](http://hyperopt.github.io/hyperopt/)
5. [Skopt](https://scikit-optimize.github.io)


### Article
1. E. Brochu,, V. M. Cora, and N. De Freitas. A tutorial on Bayesian optimization of expensive cost functions, with application to active user modeling and hierarchical reinforcement learning., arXiv preprint arXiv:1012.2599 (2010), https://arxiv.org/pdf/1012.2599.pdf.
2. J. Bergstra, R. Bardenet, Y. Bengio, Algorithms for hyper-parameter optimization., Advances in Neural Information Processing Systems, 2011, https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf.
3. J. Snoek, H. Larochelle, and R. P. Adams. Practical bayesian optimization of machine learning algorithms.. Advances in neural information processing systems, 2012, https://arxiv.org/pdf/1206.2944.pdf.
4. J. Bergstra, D. Yamins, and D. D. Cox. Making a Science of Model Search: Hyperparameter Optimization in Hundreds of Dimensions for Vision Architectures., ICML (1) 28 (2013): 115-123., https://www.jmlr.org/proceedings/papers/v28/bergstra13.pdf.
5. https://thuijskens.github.io/2016/12/29/bayesian-optimisation/
6. https://conference.scipy.org/proceedings/scipy2013/pdfs/bergstra_hyperopt.pdf